# Загрузка всех моделей

## Эмбеддинги

In [7]:
from langchain_community.embeddings import InfinityEmbeddings

emb_model_BERTA = InfinityEmbeddings(model="sergeyzh/BERTA", infinity_api_url="http://127.0.0.1:7997")
emb_model_USER2 = InfinityEmbeddings(model="deepvk/USER2-base", infinity_api_url="http://127.0.0.1:7997")
emb_model_RU_EN = InfinityEmbeddings(model="ai-forever/ru-en-RoSBERTa ", infinity_api_url="http://127.0.0.1:7997")


## LLM

In [1]:
import re
import os
from typing import Optional, Union
from deepeval.models import DeepEvalBaseLLM
from openai import OpenAI, AsyncOpenAI
from pydantic import BaseModel
import instructor

class SGlangModel(DeepEvalBaseLLM):
    def __init__(self, 
                 model_name: str, 
                 base_url: str, 
                 api_key: Optional[str] = "NET",
                 enable_thinking: bool = False): # Параметр для управления режимом мышления Qwen3
        """
        Инициализирует модель SGlang.

        Args:
            model_name (str): Имя модели.
            base_url (str): Базовый URL для API.
            api_key (Optional[str]): API ключ. По умолчанию "NET".
            enable_thinking (bool): Флаг для управления поведением моделей Qwen3.
                                     Если True, к промпту для Qwen3 будет добавлен "/think" (для стимуляции процесса размышления).
                                     Если False, к промпту для Qwen3 будет добавлен "/no_think".
                                     Вне зависимости от этого флага, блок <think> будет удален из финального вывода
                                     для Qwen3 в режиме генерации текста (когда schema is None).
                                     По умолчанию False.
        """
        self.model_name = model_name
        self.base_url = base_url
        self.api_key = api_key if api_key is not None else os.getenv("OPENAI_API_KEY")
        self.enable_thinking = enable_thinking 
        self._sync_client: Optional[OpenAI] = None
        self._async_client: Optional[AsyncOpenAI] = None

    def load_model(self) -> OpenAI:
        if self._sync_client is None:
            self._sync_client = OpenAI(base_url=self.base_url, api_key=self.api_key)
        return self._sync_client

    def load_async_model(self) -> AsyncOpenAI:
        if self._async_client is None:
            self._async_client = AsyncOpenAI(base_url=self.base_url, api_key=self.api_key)
        return self._async_client

    def _clean_qwen3_output(self, text_response: str) -> str:
        """
        Удаляет начальный блок <think>...</think> из ответов модели Qwen3.
        """
        pattern = r'^\s*<think>.*?</think>\s*'
        cleaned_response = re.sub(pattern, '', text_response, count=1, flags=re.DOTALL)
        return cleaned_response

    def generate(self, prompt: str, schema: Optional[BaseModel] = None) -> Union[str, BaseModel]:
        """
        Генерирует ответ от модели. 
        Для модели 'Qwen3' (без схемы):
        - К промпту добавляется "/think" или "/no_think" в зависимости от self.enable_thinking.
        - Блок <think> всегда удаляется из финального ответа.
        """
        client = self.load_model()
        
        processed_prompt = prompt
        # Проверка на Qwen3 (нечувствительная к регистру) и отсутствие схемы
        is_qwen3_text_mode = "qwen3" in self.model_name.lower() and schema is None

        if is_qwen3_text_mode:
            # Удаляем существующие теги /think или /no_think из конца промпта
            processed_prompt = re.sub(r'\s*/think\s*$', '', processed_prompt, flags=re.IGNORECASE).strip()
            processed_prompt = re.sub(r'\s*/no_think\s*$', '', processed_prompt, flags=re.IGNORECASE).strip()
            
            if self.enable_thinking:
                processed_prompt += " /think" # Инструктируем Qwen3 выполнить процесс размышления
            else:
                processed_prompt += " /no_think" # Инструктируем Qwen3 пропустить/минимизировать размышления
        
        try:
            if schema is None:
                response = client.chat.completions.create(
                    model=self.model_name,
                    messages=[{"role": "user", "content": processed_prompt}], 
                )
                raw_content = response.choices[0].message.content
                
                if is_qwen3_text_mode:
                    # Для Qwen3 в текстовом режиме всегда очищаем вывод от блока <think>.
                    # self.enable_thinking контролирует, получает ли модель /think или /no_think в промпте,
                    # т.е. будет ли она проходить через процесс размышления.
                    return self._clean_qwen3_output(raw_content)
                else:
                    # Для других моделей возвращаем "сырой" контент
                    return raw_content
            else:
                # Режим структурированного вывода с instructor
                instructor_client = instructor.from_openai(
                    client=client,
                    mode=instructor.Mode.JSON 
                )
                response_obj = instructor_client.chat.completions.create(
                    model=self.model_name,
                    messages=[{"role": "user", "content": prompt}], 
                    response_model=schema, 
                )
                return response_obj
        except Exception as e:
            print(f"Ошибка при синхронной генерации VLLM/Instructor для промпта '{prompt[:50]}...': {e}")
            raise e

    async def a_generate(self, prompt: str, schema: Optional[BaseModel] = None) -> Union[str, BaseModel]:
        """
        Асинхронно генерирует ответ от модели.
        Для модели 'Qwen3' (без схемы):
        - К промпту добавляется "/think" или "/no_think" в зависимости от self.enable_thinking.
        - Блок <think> всегда удаляется из финального ответа.
        """
        client = self.load_async_model()

        processed_prompt = prompt
        # Проверка на Qwen3 (нечувствительная к регистру) и отсутствие схемы
        is_qwen3_text_mode = "qwen3" in self.model_name.lower() and schema is None

        if is_qwen3_text_mode:
            # Удаляем существующие теги /think или /no_think из конца промпта
            processed_prompt = re.sub(r'\s*/think\s*$', '', processed_prompt, flags=re.IGNORECASE).strip()
            processed_prompt = re.sub(r'\s*/no_think\s*$', '', processed_prompt, flags=re.IGNORECASE).strip()

            if self.enable_thinking:
                processed_prompt += " /think" # Инструктируем Qwen3 выполнить процесс размышления
            else:
                processed_prompt += " /no_think" # Инструктируем Qwen3 пропустить/минимизировать размышления

        try:
            if schema is None:
                response = await client.chat.completions.create(
                    model=self.model_name,
                    messages=[{"role": "user", "content": processed_prompt}], 
                )
                raw_content = response.choices[0].message.content

                if is_qwen3_text_mode:
                    # Для Qwen3 в текстовом режиме всегда очищаем вывод от блока <think>.
                    # self.enable_thinking контролирует, получает ли модель /think или /no_think в промпте.
                    return self._clean_qwen3_output(raw_content)
                else:
                    # Для других моделей возвращаем "сырой" контент
                    return raw_content
            else:
                # Режим структурированного вывода с instructor
                instructor_client = instructor.from_openai(
                    client=client,
                    mode=instructor.Mode.JSON
                )
                response_obj = await instructor_client.chat.completions.create(
                    model=self.model_name,
                    messages=[{"role": "user", "content": prompt}], 
                    response_model=schema,
                )
                return response_obj
        except Exception as e:
            print(f"Ошибка при асинхронной генерации VLLM/Instructor для промпта '{prompt[:50]}...': {e}")
            raise e

    def get_model_name(self) -> str:
        return self.model_name

In [9]:
Qwen3_8_SGlang = SGlangModel(model_name="Qwen/Qwen3-8B", base_url="http://85.143.167.11:30000/v1")
Qwen3_8_SGlang_Reasoning = SGlangModel(model_name="Qwen/Qwen3-8B", base_url="http://85.143.167.11:30000/v1", enable_thinking = True)

In [10]:
from langchain_openai import ChatOpenAI

Cogito = ChatOpenAI(model="deepcogito/cogito-v1-preview-llama-8B", base_url = "http://85.143.167.11:30000/v1", api_key="NO")
Qwen3_8 = ChatOpenAI(model="Qwen/Qwen3-8B", base_url = "http://85.143.167.11:30000/v1", api_key="NO")
Gemma = ChatOpenAI(model="google/gemma-3-4b-it", base_url = "http://85.143.167.11:30000/v1", api_key="NO")
Yandex = ChatOpenAI(model="yandex/YandexGPT-5-Lite-8B-instruct", base_url = "http://85.143.167.11:30000/v1", api_key="NO")



---

## Документы + ДБ

In [ ]:
# full_text = "\n".join([doc.page_content for doc in docs])
# file_path=['/mnt/sdb1/PycharmProjects/CODUP/AI-tutor-other/docs/for_golds/Anatomia_cheloveka_1_tom_2.pdf',
#                     '/mnt/sdb1/PycharmProjects/CODUP/AI-tutor-other/docs/for_golds/Kapandzhi_-_Pozvonochnik.pdf',
#                     '/mnt/sdb1/PycharmProjects/CODUP/AI-tutor-other/docs/for_golds/Kozhnye_i_venericheskie_bolezni_pod_red_O_Yu_Olisovoi_774.pdf',
#                     '/mnt/sdb1/PycharmProjects/CODUP/AI-tutor-other/docs/for_golds/Molekulyarnaya_biologia_kletki_Tom_1.pdf']

In [2]:
from docling.document_converter import DocumentConverter, PdfFormatOption  
from docling.datamodel.pipeline_options import PdfPipelineOptions, TableFormerMode, AcceleratorOptions, AcceleratorDevice, RapidOcrOptions, EasyOcrOptions
from docling.datamodel.base_models import InputFormat
from docling_core.types.doc import ImageRefMode 

pipeline_options = PdfPipelineOptions()  
pipeline_options.do_ocr = True
pipeline_options.do_table_structure = True
pipeline_options.ocr_options = RapidOcrOptions()   
pipeline_options.table_structure_options.mode = TableFormerMode.ACCURATE  
pipeline_options.do_code_enrichment = False  
pipeline_options.do_formula_enrichment = False  
pipeline_options.do_picture_classification = False  
pipeline_options.do_picture_description = False  
pipeline_options.generate_page_images = False  
pipeline_options.generate_picture_images = False  
  
pipeline_options.accelerator_options = AcceleratorOptions(  
    num_threads=8,   
    device=AcceleratorDevice.CUDA 
)  

def clean_soft_hyphens(text):  
    """Удаляет символы мягкого переноса из текста"""  
    return text.replace('\xad ', '')  

converter = DocumentConverter(  
    format_options={  
        InputFormat.PDF: PdfFormatOption(  
            pipeline_options=pipeline_options 
        )  
    }  
)  
  
def process_pdf(pdf_path, output_path=None):  
    """  
    Обрабатывает PDF-файл и возвращает результат в формате Markdown  
      
    Args:  
        pdf_path: Путь к PDF-файлу  
        output_path: Путь для сохранения результата (опционально)  
      
    Returns:  
        Текст в формате Markdown  
    """  
    result = converter.convert(pdf_path)  
    if result.status.value == "success":   
        markdown_text = result.document.export_to_markdown(image_mode=ImageRefMode.PLACEHOLDER)
        if output_path:  
            with open(output_path, "w", encoding="utf-8") as f:  
                f.write(markdown_text)  
            print(f"Результат сохранен в {output_path}")  
          
        return markdown_text  
    else:  
        print(f"Ошибка конвертации: {result.errors}")  
        return None 

In [4]:
%%time
pdf_path = "/mnt/sdb1/PycharmProjects/CODUP/AI-tutor-other/docs/for_golds/Anatomia_cheloveka_1_tom_2-161-164.pdf"  
output_path = "../data/output_markdown/Anatomia_cheloveka_1_tom_2-161-164.md"  
      
markdown_text = process_pdf(pdf_path, output_path)

Результат сохранен в ../data/output_markdown/Anatomia_cheloveka_1_tom_2-161-164.md
CPU times: user 2min 34s, sys: 11.4 s, total: 2min 46s
Wall time: 1min 10s


In [2]:
%%time
pdf_path = "/mnt/sdb1/PycharmProjects/CODUP/AI-tutor-other/docs/for_golds/Anatomia_cheloveka_1_tom_2.pdf"  
output_path = "../data/output_markdown/Anatomia_cheloveka_1_tom_2.md"  
      
markdown_text = process_pdf(pdf_path, output_path)

Результат сохранен в ../data/output_markdown/Anatomia_cheloveka_1_tom_2.md
CPU times: user 6min 13s, sys: 6.31 s, total: 6min 19s
Wall time: 6min 25s


In [ ]:
%%time
pdf_path = "/mnt/sdb1/PycharmProjects/CODUP/AI-tutor-other/docs/for_golds/Kapandzhi_-_Pozvonochnik.pdf"  
output_path = "../data/output_markdown/Kapandzhi_-_Pozvonochnik.md"  
      
# markdown_text_2 = process_pdf(pdf_path, output_path) 
markdown_text_2 = process_pdf(pdf_path) 

Результат сохранен в ../data/output_markdown/Kapandzhi_-_Pozvonochnik.md
CPU times: user 2min 34s, sys: 321 ms, total: 2min 35s
Wall time: 2min 36s


In [7]:
%%time
pdf_path = "/mnt/sdb1/PycharmProjects/CODUP/AI-tutor-other/docs/for_golds/Kozhnye_i_venericheskie_bolezni_pod_red_O_Yu_Olisovoi_774.pdf"  
output_path = "../data/output_markdown/Kozhnye_i_venericheskie_bolezni_pod_red_O_Yu_Olisovoi_774.md"  
      
markdown_text_3 = process_pdf(pdf_path, output_path) 

Результат сохранен в ../data/output_markdown/Kozhnye_i_venericheskie_bolezni_pod_red_O_Yu_Olisovoi_774.md
CPU times: user 3min 58s, sys: 1.03 s, total: 3min 59s
Wall time: 4min 1s


In [2]:
%%time
pdf_path = "/mnt/sdb1/PycharmProjects/CODUP/AI-tutor-other/docs/for_golds/Molekulyarnaya_biologia_kletki_Tom_1.pdf"  
output_path = "../data/output_markdown/Molekulyarnaya_biologia_kletki_Tom_1.md"  
      
markdown_text_4 = process_pdf(pdf_path, output_path) 

Результат сохранен в ../data/output_markdown/Molekulyarnaya_biologia_kletki_Tom_1.md
CPU times: user 11min 28s, sys: 14.8 s, total: 11min 43s
Wall time: 12min 6s


In [ ]:
from langchain_qdrant import Qdrant    
from langchain_core.runnables import RunnablePassthrough      

vectorstore = Qdrant.from_documents(  
    documents=chunks_splited,  
    embedding=emb_model_BERTA,  
    collection_name="Med_docs",  
    url="http://localhost:6333", 
)

---

## Чанкинг

In [21]:
import requests
import numpy as np
from typing import List, Optional, Any, Callable
from chonkie.embeddings import BaseEmbeddings

class InfinityEmbeddingsChonkie(BaseEmbeddings):
    """
    Класс для работы с моделями эмбеддингов через локальный движок infinity.
    Наследуется от chonkie.embeddings.BaseEmbeddings.
    Предполагает, что numpy и chonkie установлены.
    """
    def __init__(self, model_name: str, api_url: str = "http://localhost:7997"):
        """
        Инициализирует класс.

        Args:
            model_name: Имя модели, используемой в infinity.
            api_url: Базовый URL API infinity.
        """
        # ИСПРАВЛЕНО: Убран вызов super().__init__() для обхода ошибки
        # TypeError при инициализации BaseEmbeddings в chonkie.
        # super().__init__()
        self.model_name = model_name
        self.api_url = api_url.rstrip('/')
        self._dimension = None # Кэшируем размерность

    @property
    def dimension(self) -> int:
        """Возвращает размерность эмбеддингов модели."""
        if self._dimension is None:
            try:
                dummy_embedding = self.embed("test") # Получаем np.ndarray
                self._dimension = dummy_embedding.shape[0]
            except Exception as e:
                print(f"Не удалось определить размерность автоматически: {e}")
                # Можно установить значение по умолчанию или перевыбросить ошибку
                # self._dimension = 768 # Пример
                raise ValueError("Не удалось определить размерность эмбеддингов.") from e
        return self._dimension

    def _make_request(self, endpoint: str, json_payload: dict) -> dict:
        """Отправляет запрос к API infinity."""
        try:
            response = requests.post(f"{self.api_url}/{endpoint.lstrip('/')}", json=json_payload)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException as e:
            print(f"Ошибка запроса к API infinity ({endpoint}): {e}")
            raise

    def embed(self, text: str) -> "np.ndarray":  
        """Генерирует эмбеддинг для одного текста."""  
        payload = {  
            "model": self.model_name,  
            "input": [text],  
            "encoding_format": "float",  
            "modality": "text"  
        }  
        result = self._make_request("embeddings", payload)  
        embedding_data = result.get("data", [])  
        if embedding_data and "embedding" in embedding_data[0]:  
            return np.array(embedding_data[0]["embedding"], dtype=np.float32)  
        else:  
            raise ValueError("Не удалось получить эмбеддинг из ответа API.")  
    
    def embed_batch(self, texts: List[str]) -> List["np.ndarray"]:  
        """Генерирует эмбеддинги для списка текстов."""  
        payload = {  
            "model": self.model_name,  
            "input": texts,  
            "encoding_format": "float",  
            "modality": "text"  
        }  
        result = self._make_request("embeddings", payload)  
        embedding_data = result.get("data", [])  
        embeddings = []  
        if embedding_data and all("embedding" in item for item in embedding_data):  
            for item in embedding_data:  
                embeddings.append(np.array(item["embedding"], dtype=np.float32))  
            return embeddings  
        else:  
            raise ValueError("Не удалось получить эмбеддинги из ответа API.")

    def _simple_token_counter(self, text: str) -> int:
        """Простой счетчик токенов (по словам)."""
        return len(text.split())

    # Вспомогательные методы count_tokens* не являются частью BaseEmbeddings
    def count_tokens(self, text: str) -> int:
        """Подсчитывает токены в тексте (использует простой счетчик)."""
        return self._simple_token_counter(text)

    def count_tokens_batch(self, texts: List[str]) -> List[int]:
        """Подсчитывает токены для списка текстов."""
        return [self._simple_token_counter(text) for text in texts]

    def get_tokenizer_or_token_counter(self) -> Callable[[str], int]:
        """Возвращает функцию для подсчета токенов."""
        return self._simple_token_counter

    @classmethod
    def is_available(cls, api_url: str = "http://localhost:7997") -> bool:
        """
        Проверяет доступность API infinity.
        Предполагает, что зависимости Python (numpy, chonkie, requests) установлены.
        """
        # Проверяем только доступность API infinity
        try:
            response = requests.get(f"{api_url.rstrip('/')}/models", timeout=5)
            response.raise_for_status()
            # Дополнительно можно проверить ответ, например, что это JSON
            # response.json()
            return True
        except requests.exceptions.RequestException as e:
            print(f"Сервер Infinity недоступен по адресу {api_url}: {e}")
            return False
        except Exception as e:
             print(f"Неожиданная ошибка при проверке доступности Infinity ({api_url}): {e}")
             return False


    def __repr__(self) -> str:
        """Возвращает строковое представление объекта."""
        return f"InfinityEmbeddings(model_name='{self.model_name}', api_url='{self.api_url}')"



In [4]:
import json  
import requests  
import numpy as np  
from typing import List, Optional, Any, Callable  
from chonkie.embeddings import BaseEmbeddings  
  
class InfinityEmbeddingsChonkie(BaseEmbeddings):  
    """  
    Класс для работы с моделями эмбеддингов через локальный движок infinity.  
    Наследуется от chonkie.embeddings.BaseEmbeddings.  
    """  
    def __init__(self, model_name: str, api_url: str = "http://localhost:7997"):  
        """  
        Инициализирует класс.  
  
        Args:  
            model_name: Имя модели, используемой в infinity.  
            api_url: Базовый URL API infinity.  
        """  
        # Вызываем инициализатор родительского класса  
        super().__init__()  
        self.model_name = model_name  
        self.api_url = api_url.rstrip('/')  
        self._dimension = None  
          
        # Проверяем доступность модели при инициализации  
        if not self.check_model_availability():  
            raise ValueError(f"Модель {model_name} недоступна в Infinity API по адресу {api_url}")  
  
    def check_model_availability(self):  
        """Проверяет доступность указанной модели."""  
        try:  
            response = requests.get(f"{self.api_url}/models")  
            response.raise_for_status()  
            models_data = response.json()  
            available_models = [model["id"] for model in models_data.get("data", [])]  
            if self.model_name not in available_models:  
                print(f"Модель {self.model_name} не найдена. Доступные модели: {available_models}")  
                return False  
            return True  
        except Exception as e:  
            print(f"Ошибка при проверке доступности модели: {e}")  
            return False  
  
    @property  
    def dimension(self) -> int:  
        """Возвращает размерность эмбеддингов модели."""  
        if self._dimension is None:  
            try:  
                dummy_embedding = self.embed("test")  
                self._dimension = dummy_embedding.shape[0]  
            except Exception as e:  
                print(f"Не удалось определить размерность автоматически: {e}")  
                raise ValueError("Не удалось определить размерность эмбеддингов.") from e  
        return self._dimension  
  
    def _make_request(self, endpoint: str, json_payload: dict) -> dict:  
        """Отправляет запрос к API infinity."""  
        try:  
            response = requests.post(f"{self.api_url}/{endpoint.lstrip('/')}", json=json_payload)   
            response.raise_for_status()  
            return response.json()  
        except requests.exceptions.HTTPError as e:  
            if e.response.status_code == 422:  
                error_details = e.response.json()  
                print(f"Детали ошибки валидации: {json.dumps(error_details, ensure_ascii=False)}")  
            raise  
        except requests.exceptions.RequestException as e:  
            print(f"Ошибка запроса к API infinity ({endpoint}): {e}")  
            raise  
  
    def embed(self, text: str) -> "np.ndarray":  
        """Генерирует эмбеддинг для одного текста."""  
        # Проверяем, что текст не пустой  
        if not text or not text.strip():  
            # Возвращаем нулевой вектор для пустого текста  
            if self._dimension is not None:  
                return np.zeros(self._dimension, dtype=np.float32)  
            else:  
                raise ValueError("Нельзя создать эмбеддинг для пустого текста, когда размерность неизвестна")  
          
        payload = {  
            "model": self.model_name,  
            "input": [text],  
            "encoding_format": "float",  
            "modality": "text"  
        }  
          
        result = self._make_request("embeddings", payload)  
        embedding_data = result.get("data", [])  
          
        if embedding_data and "embedding" in embedding_data[0]:  
            return np.array(embedding_data[0]["embedding"], dtype=np.float32)  
        else:  
            raise ValueError(f"Не удалось получить эмбеддинг из ответа API: {result}")  
  
    def embed_batch(self, texts: List[str]) -> List["np.ndarray"]:  
        """Генерирует эмбеддинги для списка текстов."""  
        if not texts:  
            return []  
              
        # Фильтруем пустые тексты  
        non_empty_texts = []  
        empty_indices = []  
          
        for i, text in enumerate(texts):  
            if text and text.strip():  
                non_empty_texts.append(text)  
            else:  
                empty_indices.append(i)  
          
        # Если все тексты пустые, возвращаем список нулевых векторов  
        if not non_empty_texts:  
            if self._dimension is not None:  
                return [np.zeros(self._dimension, dtype=np.float32) for _ in texts]  
            else:  
                raise ValueError("Нельзя создать эмбеддинги для пустых текстов, когда размерность неизвестна")  
          
        payload = {  
            "model": self.model_name,  
            "input": non_empty_texts,  
            "encoding_format": "float",  
            "modality": "text"  
        }  
          
        try:  
            result = self._make_request("embeddings", payload)  
            embedding_data = result.get("data", [])  
              
            if embedding_data and all("embedding" in item for item in embedding_data):  
                embeddings = [np.array(item["embedding"], dtype=np.float32) for item in embedding_data]  
                  
                # Вставляем нулевые векторы для пустых текстов  
                if empty_indices:  
                    if self._dimension is None:  
                        self._dimension = embeddings[0].shape[0]  
                      
                    full_embeddings = []  
                    embedding_idx = 0  
                      
                    for i in range(len(texts)):  
                        if i in empty_indices:  
                            full_embeddings.append(np.zeros(self._dimension, dtype=np.float32))  
                        else:  
                            full_embeddings.append(embeddings[embedding_idx])  
                            embedding_idx += 1  
                      
                    return full_embeddings  
                else:  
                    return embeddings  
            else:  
                raise ValueError(f"Не удалось получить эмбеддинги из ответа API: {result}")  
        except Exception as e:  
            print(f"Ошибка при получении эмбеддингов пакетом: {e}")  
            print("Пробуем получить эмбеддинги по одному...")  
              
            # Fallback: получаем эмбеддинги по одному  
            embeddings = []  
            for text in texts:  
                try:  
                    if text and text.strip():  
                        embeddings.append(self.embed(text))  
                    else:  
                        if self._dimension is None and embeddings:  
                            self._dimension = embeddings[0].shape[0]  
                        elif self._dimension is None:  
                            # Если это первый текст и он пустой, получаем размерность из непустого текста  
                            for t in texts:  
                                if t and t.strip():  
                                    dummy_emb = self.embed(t)  
                                    self._dimension = dummy_emb.shape[0]  
                                    break  
                          
                        embeddings.append(np.zeros(self._dimension, dtype=np.float32))  
                except Exception as inner_e:  
                    print(f"Ошибка при получении эмбеддинга для текста '{text[:50]}...': {inner_e}")  
                    raise  
              
            return embeddings  
  
    def similarity(self, u: "np.ndarray", v: "np.ndarray") -> float:  
        """Вычисляет косинусное сходство между двумя векторами."""  
        dot_product = np.dot(u, v)  
        norm_u = np.linalg.norm(u)  
        norm_v = np.linalg.norm(v)  
          
        # Избегаем деления на ноль  
        if norm_u == 0 or norm_v == 0:  
            return 0.0  
              
        return float(dot_product / (norm_u * norm_v))  
  
    def _simple_token_counter(self, text: str) -> int:  
        """Простой счетчик токенов (по словам)."""  
        return len(text.split())  
  
    def count_tokens(self, text: str) -> int:  
        """Подсчитывает токены в тексте (использует простой счетчик)."""  
        return self._simple_token_counter(text)  
  
    def count_tokens_batch(self, texts: List[str]) -> List[int]:  
        """Подсчитывает токены для списка текстов."""  
        return [self._simple_token_counter(text) for text in texts]  
  
    def get_tokenizer_or_token_counter(self) -> Callable[[str], int]:  
        """Возвращает функцию для подсчета токенов."""  
        return self._simple_token_counter  
  
    @classmethod  
    def is_available(cls, api_url: str = "http://localhost:7997") -> bool:  
        """Проверяет доступность API infinity."""  
        try:  
            response = requests.get(f"{api_url.rstrip('/')}/models", timeout=5)  
            response.raise_for_status()  
            return True  
        except Exception as e:  
            print(f"Сервер Infinity недоступен по адресу {api_url}: {e}")  
            return False  
  
    def __repr__(self) -> str:  
        """Возвращает строковое представление объекта."""  
        return f"InfinityEmbeddingsChonkie(model_name='{self.model_name}', api_url='{self.api_url}')"

In [2]:
embed_chonkie = InfinityEmbeddingsChonkie(model_name="sergeyzh/BERTA", api_url="http://127.0.0.1:7997")

In [14]:
from chonkie import RecursiveChunker  
  
chunker = RecursiveChunker(  
    chunk_size= 300,  
    min_characters_per_chunk=50,
    tokenizer_or_token_counter = "sergeyzh/BERTA"
)  
  
chunks = chunker.chunk(markdown_doc_med)

In [3]:
file_path = "../data/output_markdown/Kapandzhi_-_Pozvonochnik.md" 

with open(file_path, 'r', encoding='utf-8') as file:
    markdown_doc_med = file.read()

In [3]:
file_path = "/home/lanarich/Рабочий стол/Diploma/data/output_markdown/Anatomia_cheloveka_1_tom_2-52-57.md" 

with open(file_path, 'r', encoding='utf-8') as file:
    markdown_doc_med_small = file.read()

In [46]:
from chonkie import SDPMChunker  
  
chunker = SDPMChunker(
    embedding_model=embed_chonkie,          # Твоя модель эмбеддингов
    threshold='auto',                       # Автоматический подбор порога схожести. Хороший старт, т.к. плотность тем в учебнике может меняться.
                                            # Альтернативно: попробуй число от 0.4 до 0.6 или процентиль '90%' или '95%', если 'auto' не устроит.
    chunk_size=768,                         # Увеличим максимальный размер чанка по сравнению с дефолтом (512).
                                            # Учебники могут содержать длинные абзацы или описания, требующие большего контекста.
                                            # Подбирай в зависимости от модели, которая будет использовать чанки (ее лимит токенов). 1024 тоже можно рассмотреть.
    min_sentences=2,                        # Минимальное количество предложений в чанке. Увеличим с 1 до 2, чтобы избежать слишком коротких чанков, особенно если есть короткие заголовки/подписи.
    skip_window=2,                          # Количество чанков (групп), через которые будет искаться семантическая связь.
                                            # Увеличим с 1 до 2. В учебниках связанные понятия могут быть разделены парой абзацев или небольшим подразделом.
                                            # Слишком большое значение может объединить несвязанные темы.
    min_chunk_size=100,                     # Минимальный размер чанка в токенах. Увеличим с дефолтных 2 до 100.
                                            # Это критично, чтобы отсечь очень короткие, бессодержательные чанки (например, из отдельных строк таблицы или коротких заголовков).
    min_characters_per_sentence=15,         # Немного увеличим минимальную длину предложения, чтобы отфильтровать совсем короткие строки, которые могут не нести смысла сами по себе.
    delim=['.', '!', '?', '\n', '\n\n'],    # Явно добавим '\n\n' как разделитель. Это может помочь лучше отделять абзацы и блоки перед/после таблиц/заголовков.
                                            # Важно наличие '\n', так как строки таблицы часто разделены именно им.
    # Остальные параметры можно оставить по умолчанию или настроить при необходимости:
    # mode='window',                        # Режим группировки, дефолт 'window' обычно подходит.
    # similarity_window=1,                  # Окно для расчета схожести при 'auto' threshold. Дефолт 1 обычно достаточен.
    # threshold_step=0.01,                  # Шаг для расчета порога. Дефолт нормальный.
    # include_delim='prev',                 # Включать ли разделитель. Дефолт 'prev' окей.
    # return_type='chunks'                  # Возвращать объекты SemanticChunk с метаданными (рекомендуется).
)

chunks_small = chunker.chunk(markdown_doc_med_small)

In [52]:
from chonkie import SemanticChunker

chunker_semantic = SemanticChunker(
    embedding_model=embed_chonkie, # Твоя модель
    threshold=0.7,              # Или число/процентиль
    chunk_size=1000,                # Или 512/1024
    min_sentences=2,               # Минимум предложений
    min_chunk_size=100,            # **Обязательно установи!**
    delim=['.', '!', '?', '\n', '\n\n'], # Важные разделители
    min_characters_per_sentence=15 # Можно тоже поднять
)
chunks_semantic = chunker_semantic.chunk(markdown_doc_med_small)

In [53]:
for i in chunks_semantic:
    print(i.text, "\n", "<!!!>") 
 

<!-- image -->

Рис. 32. Расположение костных перекладин (балок) в губчатом веществе по линии сжатия и растяжения:

1   - линии сжатия; 2 - линии растяжения.

1

Рис. 33. Строение диафиза трубчатой кости и положение его надкостницы:

<!-- image -->

1   - надкостница; 2 - компактное вещество; 3 - костномозговая полость. 
 <!!!>


которых кость растет в толщину и регенерирует после повреждения. Таким образом, надкостница выполняет не только защитную и трофическую функции, но и, подобно эндосту, участвует в костеобразовании.

Кости отличаются значительной пластичностью. Их форма может изменяться под действием физических нагрузок, что связано с увеличением или уменьшением количества остеонов, изменением толщины костных пластинок компактного и губчатого вещества. 
 <!!!>
 Для оптимального развития кости наиболее предпочтительны умеренные регулярные физические нагрузки.  Сидячий образ жизни и пониженная физическая активность способствуют ослаблению и истончению кости.  Кость приобретает кру

In [2]:
from chonkie import RecursiveChunker


chunker = RecursiveChunker.from_recipe(
        name="markdown",
        # lang="en", # Можно попробовать и "en", если без lang тоже ошибка
        chunk_size=768, # Или 512 / 1024
        tokenizer_or_token_counter="sergeyzh/BERTA", # Или токенизатор твоей модели эмбеддингов
        min_characters_per_chunk=75 # Увеличим значение по умолчанию (24)
)




In [4]:
chunks_rec = chunker.chunk(markdown_doc_med_small)

In [8]:
chunks_rec[0].text.strip()

'<!-- image -->\n\nРис. 32. Расположение костных перекладин (балок) в губчатом веществе по линии сжатия и растяжения:\n\n1   - линии сжатия; 2 - линии растяжения.\n\n1\n\nРис. 33. Строение диафиза трубчатой кости и положение его надкостницы:\n\n<!-- image -->\n\n1   - надкостница; 2 - компактное вещество; 3 - костномозговая полость.\n\nкоторых кость растет в толщину и регенерирует после повреждения. Таким образом, надкостница выполняет не только защитную и трофическую функции, но и, подобно эндосту, участвует в костеобразовании.\n\nКости отличаются значительной пластичностью. Их форма может изменяться под действием физических нагрузок, что связано с увеличением или уменьшением количества остеонов, изменением толщины костных пластинок компактного и губчатого вещества. Для оптимального развития кости наиболее предпочтительны умеренные регулярные физические нагрузки.  Сидячий образ жизни и пониженная физическая активность способствуют ослаблению и истончению кости.  Кость приобретает круп

In [60]:
for i in chunks_rec:
    print(i.text, "\n", "<!!!>")

<!-- image -->

## Глава 1

## ПОЗВОНОЧНЫЙ СТОЛБ В ЦЕЛОМ

## ЧЕЛОВЕК -  ЭТО  ПОЗВОНОЧНОЕ

Человек  как  вид  является  позвоночным.  Он  представляет  собой  результат  длинного  пути  эволюции, начиная  с  того  момента,  когда рыбы покинули  море и  начали заселять сушу.

В  ходе  эволюции  в  строении  позвоночника  происходили глубокие изменения, но он  всегда состоит из коротких  костей,  вставленных  друг  в  друга  и  подвижных  по отношению друг другу, - позвонков.

Скелет  человека, основой  которого  является позвоночник,  -  это  результат  трансформации  скелета  древней  костной  рыбы (crossopterygien) в  скелет животного  с  четырьмя  лапами  и  хвостом,  промежуточную  форму  между  рыбой  и  рептилией.  Все элементы  этой  исходной  модели  можно  найти  и  в скелете  человека,  более  или  менее  измененные,  но обладающие двумя  важными  характеристиками:

- · Исчезновение  хвоста.
- · Переход в  вертикальное  положение.

Этот  костно-суставной  комплекс  служит  для 

In [77]:
from chonkie import LateChunker
# Попробуй lang="ru", если не сработает - без lang или lang="en"
chunker = LateChunker.from_recipe(
        name = "markdown",
        embedding_model="sergeyzh/BERTA",
        chunk_size=300, # Или 512 / 1024 (исходя из лимита LLM)
        min_characters_per_chunk=75 # Увеличь с дефолта 24
)

chunks_late = chunker.chunk(markdown_doc_med)

Default prompt name is set to 'Classification'. This prompt will be applied to all `encode()` calls, except if `encode()` is called with `prompt` or `prompt_name` parameters.
Token indices sequence length is longer than the specified maximum sequence length for this model (515 > 512). Running this sequence through the model will result in indexing errors
/home/lanarich/.cache/pypoetry/virtualenvs/diploma-au9TFMuk-py3.11/lib/python3.11/site-packages/numpy/_core/fromnumeric.py:3860: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/lanarich/.cache/pypoetry/virtualenvs/diploma-au9TFMuk-py3.11/lib/python3.11/site-packages/numpy/_core/_methods.py:137: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(


In [78]:
for i in chunks_late:
    print(i.text, "\n", "<!!!>")

<!-- image -->

## Глава 1

## ПОЗВОНОЧНЫЙ СТОЛБ В ЦЕЛОМ

## ЧЕЛОВЕК -  ЭТО  ПОЗВОНОЧНОЕ

Человек  как  вид  является  позвоночным.  Он  представляет  собой  результат  длинного  пути  эволюции, начиная  с  того  момента,  когда рыбы покинули  море и  начали заселять сушу.

В  ходе  эволюции  в  строении  позвоночника  происходили глубокие изменения, но он  всегда состоит из коротких  костей,  вставленных  друг  в  друга  и  подвижных  по отношению друг другу, - позвонков.

Скелет  человека, основой  которого  является позвоночник,  -  это  результат  трансформации  скелета  древней  костной  рыбы (crossopterygien) в  скелет животного  с  четырьмя  лапами  и  хвостом,  промежуточную  форму  между  рыбой  и  рептилией.  Все элементы  этой  исходной  модели  можно  найти  и  в скелете  человека,  более  или  менее  измененные,  но обладающие двумя  важными  характеристиками:

- · Исчезновение  хвоста.
- · Переход в  вертикальное  положение.

Этот  костно-суставной  комплекс  служит  для 

---
## Retrivers

NAIVE

In [ ]:
from deepeval.dataset import EvaluationDataset  

dataset_name = "30 медицинских вопросов короткие учебники"
dataset = EvaluationDataset()  
dataset.pull(dataset_name)

Output()

In [44]:
import asyncio  
from deepeval.test_case import LLMTestCase  
from deepeval.metrics import ContextualRelevancyMetric  
from deepeval.evaluate import AsyncConfig   
from deepeval import evaluate  
  
async def process_goldens(dataset, retriever, max_concurrent=20):  
    semaphore = asyncio.Semaphore(max_concurrent)  
    test_cases = []  
      
    async def process_single_golden(golden):  
        async with semaphore:  
            query = golden.input  
            retrieval_results = await retriever.ainvoke(query)  
            retrieval_context = [doc.page_content for doc in retrieval_results]  
              
            return LLMTestCase(  
                input=query,  
                actual_output="",  
                retrieval_context=retrieval_context  
            )  
       
    tasks = [process_single_golden(golden) for golden in dataset.goldens]  
    test_cases = await asyncio.gather(*tasks)  
      
    return test_cases

In [45]:
test_cases = await process_goldens(dataset, retriever)  

In [52]:
dataset_naive = EvaluationDataset(test_cases=test_cases)
dataset_naive.push("Наивный ретривер 30 вопросов", auto_convert_test_cases_to_goldens=True)

✅ Dataset successfully pushed to Confident AI! View at 
]8;id=335288;https://app.confident-ai.com/project/cm9sl70hj0evpweys5yehloy4/datasets/cmafctr1101yotb487a5ofvxp\https://app.confident-ai.com/project/cm9sl70hj0evpweys5yehloy4/datasets/cmafctr1101yotb487a5ofvxp]8;;\

Gtk-Message: 15:39:30.667: Failed to load module "canberra-gtk-module"
Gtk-Message: 15:39:30.667: Failed to load module "pk-gtk-module"
Gtk-Message: 15:39:30.671: Failed to load module "canberra-gtk-module"
Gtk-Message: 15:39:30.671: Failed to load module "pk-gtk-module"


Окно или вкладка откроются в текущем сеансе браузера.


In [56]:
dataset_naive.save_as(file_type="csv", directory="../data/Retrievers", include_test_cases=True)

Evaluation dataset saved at ../data/Retrievers/20250508_154348.csv!


'../data/Retrievers/20250508_154348.csv'

In [47]:
contextual_relevancy = ContextualRelevancyMetric(model=Qwen3_8_SGlang, async_mode=True)  
results = evaluate(  
    test_cases=test_cases,  
    metrics=[contextual_relevancy],  
    async_config=AsyncConfig(),  
    hyperparameters={  
        "Ретривер": "Наивный",  
        "model": "Qwen-3-8B",  
        "top_k": 5  
    }  
)  

✨ You're running DeepEval's latest Contextual Relevancy Metric! (using Qwen/Qwen3-8B, strict=False, 
async_mode=True)...

Evaluating 30 test case(s) in parallel: |██████████|100% (30/30) [Time Taken: 02:54,  5.82s/test case]




Metrics Summary

  - ❌ Contextual Relevancy (score: 0.07142857142857142, threshold: 0.5, strict: False, evaluation model: Qwen/Qwen3-8B, reason: The score is 0.07 because the retrieval context overwhelmingly contains irrelevant information about head mobility, field of vision, and skull structures, with only two statements (out of 22) directly addressing the cervical vertebrae position in dogs and the occipital foramen location compared to humans. The relevant statements mention the horizontal positioning of the cervical spine in dogs leading to a posteriorly located occipital foramen, while humans have a more anteriorly positioned one, but these are minor and overshadowed by the majority of irrelevant content., error: None)

For test case:

  - input: Как положение шейного отдела у собак влияет на расположение затылочного отверстия по сравнению с человеком?
  - actual output: 
  - expected output: None
  - context: None
  - retrieval context: ['Череп  гоминидов,  в  частности  высши

✓ Tests finished 🎉! View results on 
]8;id=611332;https://app.confident-ai.com/project/cm9sl70hj0evpweys5yehloy4/evaluation/test-runs/cmafcbw8c01to4h4o9wos3m3l/test-cases\https://app.confident-ai.com/project/cm9sl70hj0evpweys5yehloy4/evaluation/test-runs/cmafcbw8c01to4h4o9wos3m3l/test-]8;;\
]8;id=611332;https://app.confident-ai.com/project/cm9sl70hj0evpweys5yehloy4/evaluation/test-runs/cmafcbw8c01to4h4o9wos3m3l/test-cases\cases]8;;\.

Gtk-Message: 15:25:38.456: Failed to load module "canberra-gtk-module"
Gtk-Message: 15:25:38.457: Failed to load module "pk-gtk-module"
Gtk-Message: 15:25:38.461: Failed to load module "canberra-gtk-module"
Gtk-Message: 15:25:38.461: Failed to load module "pk-gtk-module"


Окно или вкладка откроются в текущем сеансе браузера.


In [ ]:
from deepeval.dataset import EvaluationDataset  
from deepeval.test_case import LLMTestCase  
  
# Создаем тестовые случаи  
test_case1 = LLMTestCase(  
    input="Ваш запрос 1",  
    actual_output="Ответ на запрос 1",  
    retrieval_context=["Контекст 1", "Контекст 2"]  
)  
  
test_case2 = LLMTestCase(  
    input="Ваш запрос 2",  
    actual_output="Ответ на запрос 2",  
    retrieval_context=["Контекст 3", "Контекст 4"]  
)  
  
# Создаем датасет и добавляем тестовые случаи  
dataset = EvaluationDataset(test_cases=[test_case1, test_case2])  
  
# Сохраняем датасет в CSV файл  
dataset.save_as(file_type="csv", directory="./my_test_cases", include_test_cases=True)  
  
# Или в JSON файл  
dataset.save_as(file_type="json", directory="./my_test_cases", include_test_cases=True)

from deepeval.dataset import EvaluationDataset  
  
# Создаем пустой датасет  
dataset = EvaluationDataset()  
  
# Загружаем тестовые случаи из JSON файла  
dataset.add_test_cases_from_json_file(  
    file_path="./my_test_cases/test_cases.json",  
    input_key_name="input",  
    actual_output_key_name="actual_output",  
    retrieval_context_key_name="retrieval_context"  
)  
  
# Или из CSV файла  
dataset.add_test_cases_from_csv_file(  
    file_path="./my_test_cases/test_cases.csv",  
    input_col_name="input",  
    actual_output_col_name="actual_output",  
    retrieval_context_col_name="retrieval_context",  
    retrieval_context_col_delimiter=";"  
)

In [ ]:
import re
def format_docs(docs):    
    return "\n\n".join(doc.page_content for doc in docs)

def clean_qwen3_output(text_response: str) -> str:

    pattern = r'^\s*<think>.*?</think>\s*'
    cleaned_response = re.sub(pattern, '', text_response, count=1, flags=re.DOTALL)
    return cleaned_response    

# Создание промпта    
prompt = ChatPromptTemplate.from_template("""    
Ответь на вопрос, основываясь только на следующем контексте:    
{context}    
    
Вопрос: {question} /no_think  
""")    
    
rag_chain = (    
    {"context": retriever | format_docs, "question": RunnablePassthrough()}    
    | prompt    
    | Qwen_3   
    | StrOutputParser()
    | clean_qwen3_output     
)    
  
# Пример использования    
response = rag_chain.invoke("Когда и какие добавочные центры окостенения образуются в эпифизах трубчатых костей, и как они влияют на формирование отростков, бугров и гребней?")    
print(response)

## Zero-shot

## Наивный раг

In [24]:
import re
def format_docs(docs):    
    return "\n\n".join(doc.page_content for doc in docs)

def clean_qwen3_output(text_response: str) -> str:

    pattern = r'^\s*<think>.*?</think>\s*'
    cleaned_response = re.sub(pattern, '', text_response, count=1, flags=re.DOTALL)
    return cleaned_response    

# Создание промпта    
prompt = ChatPromptTemplate.from_template("""    
Ответь на вопрос, основываясь только на следующем контексте:    
{context}    
    
Вопрос: {question} /no_think  
""")    
    
rag_chain = (    
    {"context": retriever | format_docs, "question": RunnablePassthrough()}    
    | prompt    
    | Qwen_3   
    | StrOutputParser()
    | clean_qwen3_output     
)    
  
# Пример использования    
response = rag_chain.invoke("Когда и какие добавочные центры окостенения образуются в эпифизах трубчатых костей, и как они влияют на формирование отростков, бугров и гребней?")    
print(response)

Добавочные центры окостенения в эпифизах трубчатых костей образуются в самом конце внутриутробной жизни и некоторое время после рождения, а именно до 17–18 лет. Эти добавочные центры участвуют в формировании отростков, бугров и гребней костей. За счет их активности происходит развитие этих структур, что способствует окончательной форме и анатомической специфичности костей.


In [13]:
from deepeval.dataset import EvaluationDataset  
  
dataset = EvaluationDataset()  
dataset.pull("30 медицинских вопросов короткие учебники")

Output()

In [27]:
import asyncio  
from deepeval.dataset import EvaluationDataset  
from deepeval.test_case import LLMTestCase  
from deepeval.metrics import AnswerRelevancyMetric, FaithfulnessMetric, ContextualRelevancyMetric  
from deepeval import evaluate  
from tqdm.notebook import tqdm 

 
async def process_golden(golden, retriever, rag_chain):  
    """Обрабатывает один голден и создает тестовый пример"""  
    question = golden.input  
    retrieved_docs = await retriever.ainvoke(question)
    retrieval_context = [doc.page_content for doc in retrieved_docs]  
    actual_output = await rag_chain.ainvoke(question)  
       
    return LLMTestCase(  
        input=question,  
        actual_output=actual_output,  
        retrieval_context=retrieval_context,  
        expected_output=golden.expected_output if hasattr(golden, 'expected_output') else None  
    )  
  
async def evaluate_dataset(dataset_name, retriever, rag_chain):  
    """Оценивает датасет с использованием RAG Triad"""  
    dataset = EvaluationDataset()  
    dataset.pull(dataset_name)  
      
    print(f"Загружено {len(dataset.goldens)} голденов из датасета {dataset_name}")  
    print("Обработка голденов...")  
    tasks = [process_golden(golden, retriever, rag_chain) for golden in dataset.goldens]  
       
    test_cases = []  
    for task in tqdm(asyncio.as_completed(tasks), total=len(tasks), desc="Обработка голденов"):  
        test_case = await task  
        test_cases.append(test_case)  
      
    print("Настройка метрик...")  
    answer_relevancy = AnswerRelevancyMetric(async_mode=True)  
    faithfulness = FaithfulnessMetric(async_mode=True)  
    contextual_relevancy = ContextualRelevancyMetric(async_mode=True)  
       
    print("Запуск оценки...")  
    results = evaluate(  
        test_cases=test_cases,  
        metrics=[answer_relevancy, faithfulness, contextual_relevancy],  
        run_async=True  
    )  
      
    return test_cases, results  

In [ ]:
dataset_name = "30 медицинских вопросов короткие учебники"  
test_cases, results = await evaluate_dataset(dataset_name, retriever, rag_chain)  

Output()

In [ ]:
async def process_golden(golden, retriever, rag_chain):  
    """Обрабатывает один голден и создает тестовый пример"""  
    question = golden.input  
    retrieved_docs = await retriever.ainvoke(question)
    retrieval_context = [doc.page_content for doc in retrieved_docs]  
    actual_output = await rag_chain.ainvoke(question)  
       
    return LLMTestCase(  
        input=question,  
        actual_output=actual_output,  
        retrieval_context=retrieval_context,  
        expected_output=golden.expected_output if hasattr(golden, 'expected_output') else None  
    )  
  
async def evaluate_dataset(dataset_name, retriever, rag_chain):  
    """Оценивает датасет с использованием RAG Triad"""  
    dataset = EvaluationDataset()  
    dataset.pull(dataset_name)  
      
    print(f"Загружено {len(dataset.goldens)} голденов из датасета {dataset_name}")  
    print("Обработка голденов...")  
    tasks = [process_golden(golden, retriever, rag_chain) for golden in dataset.goldens]  
       
    test_cases = []  
    for task in tqdm(asyncio.as_completed(tasks), total=len(tasks), desc="Обработка голденов"):  
        test_case = await task  
        test_cases.append(test_case)  
      
    print("Настройка метрик...")  
    answer_relevancy = AnswerRelevancyMetric(async_mode=True)  
    faithfulness = FaithfulnessMetric(async_mode=True)  
    contextual_relevancy = ContextualRelevancyMetric(async_mode=True)  
       
    print("Запуск оценки...")  
    results = evaluate(  
        test_cases=test_cases,  
        metrics=[answer_relevancy, faithfulness, contextual_relevancy],  
        run_async=True  
    )  
      
    return test_cases, results 

In [19]:
async def process_golden(golden, retriever, rag_chain):  
    """Обрабатывает один голден и создает тестовый пример"""  
    question = golden.input  
    retrieved_docs = await retriever.ainvoke(question)
    retrieval_context = [doc.page_content for doc in retrieved_docs]  
    actual_output = await rag_chain.ainvoke(question)  
       
    return LLMTestCase(  
        input=question,  
        actual_output=actual_output,  
        retrieval_context=retrieval_context,  
        expected_output=golden.expected_output if hasattr(golden, 'expected_output') else None  
    ) 

In [18]:
dataset.goldens[0].input

'Каков химический состав кости, включая минеральные и органические компоненты, и как они влияют на ее структуру и прочность?'

In [12]:
[print(golden) for golden in dataset.goldens]

[]

In [25]:
import asyncio  
from deepeval.dataset import EvaluationDataset  
from deepeval.test_case import LLMTestCase  
from deepeval.metrics import AnswerRelevancyMetric, FaithfulnessMetric, ContextualRelevancyMetric  
from deepeval import evaluate  
from tqdm.notebook import tqdm 

tasks = [process_golden(golden, retriever, rag_chain) for golden in dataset.goldens]

In [27]:
test_cases = []  
for task in tqdm(asyncio.as_completed(tasks), total=len(tasks), desc="Обработка голденов"):  
    test_case = await task  
    test_cases.append(test_case) 

Обработка голденов:   0%|          | 0/30 [00:00<?, ?it/s]

In [33]:
answer_relevancy = AnswerRelevancyMetric(model= Qwen3_30, async_mode=True)  
faithfulness = FaithfulnessMetric(model= Qwen3_30, async_mode=True)  
contextual_relevancy = ContextualRelevancyMetric(model= Qwen3_30, async_mode=True)  
       
print("Запуск оценки...")  
results = evaluate(  
        test_cases=test_cases,  
        metrics=[answer_relevancy, faithfulness, contextual_relevancy]  
)

Запуск оценки...


✨ You're running DeepEval's latest Answer Relevancy Metric! (using qwen3-30-lmstudio, strict=False, 
async_mode=True)...

✨ You're running DeepEval's latest Faithfulness Metric! (using qwen3-30-lmstudio, strict=False, 
async_mode=True)...

✨ You're running DeepEval's latest Contextual Relevancy Metric! (using qwen3-30-lmstudio, strict=False, 
async_mode=True)...

Evaluating 30 test case(s) in parallel: |██████████|100% (30/30) [Time Taken: 21:49, 43.64s/test case]




Metrics Summary

  - ✅ Answer Relevancy (score: 1.0, threshold: 0.5, strict: False, evaluation model: qwen3-30-lmstudio, reason: The score is 1.00 because the response directly and accurately addressed how the cervical spine position affects the location of the foramen magnum in dogs compared to humans, with no irrelevant statements., error: None)
  - ✅ Faithfulness (score: 1.0, threshold: 0.5, strict: False, evaluation model: qwen3-30-lmstudio, reason: The score is 1.00 because there are no contradictions, and the actual output perfectly aligns with the retrieval context., error: None)
  - ✅ Contextual Relevancy (score: 0.6363636363636364, threshold: 0.5, strict: False, evaluation model: qwen3-30-lmstudio, reason: The score is 0.64 because the retrieval context includes some relevant information about the position of the cervical spine and the occipital hole in dogs and humans, but also contains unrelated statements about head rotation and sensory advantages., error: None)

For test

✓ Tests finished 🎉! View results on 
]8;id=189982;https://app.confident-ai.com/project/cm9sl70hj0evpweys5yehloy4/evaluation/test-runs/cmactndet05ep86s5oo85pmz2/test-cases\https://app.confident-ai.com/project/cm9sl70hj0evpweys5yehloy4/evaluation/test-runs/cmactndet05ep86s5oo85pmz2/test-]8;;\
]8;id=189982;https://app.confident-ai.com/project/cm9sl70hj0evpweys5yehloy4/evaluation/test-runs/cmactndet05ep86s5oo85pmz2/test-cases\cases]8;;\.

Gtk-Message: 21:07:08.180: Failed to load module "canberra-gtk-module"
Gtk-Message: 21:07:08.180: Failed to load module "pk-gtk-module"
Gtk-Message: 21:07:08.183: Failed to load module "canberra-gtk-module"
Gtk-Message: 21:07:08.183: Failed to load module "pk-gtk-module"


Окно или вкладка откроются в текущем сеансе браузера.


In [44]:
results.test_results

[TestResult(name='test_case_6', success=True, metrics_data=[MetricData(name='Answer Relevancy', threshold=0.5, success=True, score=1.0, reason='The score is 1.00 because the response directly and accurately addressed how the cervical spine position affects the location of the foramen magnum in dogs compared to humans, with no irrelevant statements.', strict_mode=False, evaluation_model='qwen3-30-lmstudio', error=None, evaluation_cost=None, verbose_logs='Statements:\n[\n    "Положение шейного отдела у собак, который почти горизонтальный из-за хождения на четырёх конечностях, приводит к нижнезаднему расположению затылочного отверстия.",\n    "В отличие от этого, у человека, чей шейный отдел имеет вертикальное положение, затылочное отверстие находится в передненижнем положении, непосредственно под черепной коробкой."\n] \n \nVerdicts:\n[\n    {\n        "verdict": "yes",\n        "reason": null\n    },\n    {\n        "verdict": "yes",\n        "reason": null\n    }\n]'), MetricData(name=

In [56]:
import pandas as pd

print("\nРезультаты оценки:")  
print(f"Количество тестовых примеров: {len(results.test_results)}")  
  
# Создание DataFrame с результатами  
scores = {  
    'AnswerRelevancy': [tc.metrics_data[0].score for tc in results.test_results],  
    'Faithfulness': [tc.metrics_data[1].score for tc in results.test_results],  
    'ContextualRelevancy': [tc.metrics_data[2].score for tc in results.test_results]  
}  
  
df = pd.DataFrame(scores)  
  
# Вычисление средних значений по каждой метрике  
metric_means = df.mean()  
  
print("\nСредние значения по метрикам:")  
for metric, mean in metric_means.items():  
    print(f"{metric}: {mean:.2f}")  
  
# Добавление общего среднего значения  
df['Average'] = df.mean(axis=1)  
print(f"\nОбщее среднее значение по всем метрикам: {metric_means.mean():.2f}")


Результаты оценки:
Количество тестовых примеров: 30

Средние значения по метрикам:
AnswerRelevancy: 0.93
Faithfulness: 0.89
ContextualRelevancy: 0.70

Общее среднее значение по всем метрикам: 0.84


In [ ]:
results.test_results[0].metrics_data[0].name

TypeError: 'MetricData' object is not subscriptable